<a href="https://colab.research.google.com/github/Sebastian009/03MAIR---Algoritmos-de-Optimizacion---2023/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos(V2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Juan Sebastian Casallas Estrella  <br>
Url: https://github.com/Sebastian009/03MAIR---Algoritmos-de-Optimizacion---2023<br>
Google Colab: https://colab.research.google.com/drive/xxxxxxxxxxxxxxxx <br>
Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Configuración de Tribunales

Descripción del problema:(copiar enunciado)

....







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

In [49]:
#%pip install PuLP
#Librerias
import pandas as pd
import numpy as np
from pulp import LpVariable, LpProblem, LpMaximize, lpSum, value, LpStatus

# Crear un problema de optimización
problema = LpProblem("Organizar los horarios de partidos de La Liga", LpMaximize)

# 1) Definir los Conjuntos y Parametros

# 1.1) Definir la informacion de los equipos
Categorias = ["A", "B", "C"]
Equipos_categoria = {'A': 3, 'B': 11, 'C': 6}
Audiencia = {'AA': 2.00, 'AB': 1.30, 'AC': 1.00,
             'BA': 1.30, 'BB': 0.90, 'BC': 0.75,
             'CA': 1.00, 'CB': 0.75, 'CC': 0.47}

# 1.2) Definir días y horarios disponibles
Dias = ['Viernes', 'Sábado', 'Domingo', 'Lunes']
Horarios = {'Viernes': [20],
            'Sábado': [12, 16, 18, 20],
            'Domingo': [12, 16, 18, 20],
            'Lunes': [20]}

# 1.3) Coeficiente de disminución segun la franja horaria
Coeficientes = {
    'Viernes': {12: 0.00, 16: 0.00, 18: 0.00, 20: 0.40},
    'Sábado':  {12: 0.55, 16: 0.70, 18: 0.80, 20: 1.00},
    'Domingo': {12: 0.45, 16: 0.75, 18: 0.85, 20: 1.00},
    'Lunes':   {12: 0.00, 16: 0.00, 18: 0.00, 20: 0.40}
}

# 1.4) Coeficiente de coincidencias
Coincidencias = {0:0, 1:0.25, 2:0.45, 3:0.60, 4:0.70, 5:0.75, 6:0.78, 7:0.80, 8:0.80}



# 2) Variables de decisión

'''
Se enumeran todas las posibles soluciones y no se tienen encuentas los partidos en donde
el equipo 1 y el equipo 2 son el mismo ademas de quitar duplicidades en las combinaciones como A1 vs B5 = B5 va A1
'''
partidos = [(eq1, num_eq1, 
             eq2, num_eq2, 
             dia, horario) for eq1 in Categorias
            for num_eq1 in range(1, Equipos_categoria[eq1]+ 1)
            for eq2 in Categorias
            for num_eq2 in range(1, Equipos_categoria[eq2]+ 1)
            if (Categorias.index(eq1) <= Categorias.index(eq2)) and ((eq1 != eq2) or (num_eq1 != num_eq2)) and ((eq1 < eq2) or (eq1 == eq2 and num_eq1 < num_eq2))
            for dia in Dias
            for horario in Horarios[dia]]


#Indica si el partido se juega o no para cada posible combinación
x = LpVariable.dicts("Partido", partidos, cat='Binary')

horarios_partidos = [(dia, horario) for dia in Dias for horario in Horarios[dia]]
y = LpVariable.dicts("PenalizacionHoraria", horarios_partidos, lowBound=0, upBound=8, cat='Continuous')


# 3) Función objetivo
problema += lpSum((Coeficientes[dia][horario] * 
                   Audiencia[eq1 + eq2] * 
                   x[(eq1, num_eq1, eq2, num_eq2, dia, horario)])
                  for eq1 in Categorias
                  for num_eq1 in range(1, Equipos_categoria[eq1]+ 1)
                  for eq2 in Categorias
                  for num_eq2 in range(1, Equipos_categoria[eq2]+ 1)
                  if (Categorias.index(eq1) <= Categorias.index(eq2)) and ((eq1 != eq2) or (num_eq1 != num_eq2)) and ((eq1 < eq2) or (eq1 == eq2 and num_eq1 < num_eq2))
                  for dia in Dias
                  for horario in Horarios[dia])


# 4) Restricciones

# 4.1) Cada equipo debe jugar al menos una unica vez
for eq1 in Categorias:
    for num_eq1 in range(1, Equipos_categoria[eq1] + 1):
        problema += (lpSum(x[(eq1, num_eq1, eq2, num_eq2, dia, horario)]
                          for eq2 in Categorias
                          for num_eq2 in range(1, Equipos_categoria[eq2] + 1)
                          if (Categorias.index(eq1) <= Categorias.index(eq2)) and ((eq1 != eq2) or (num_eq1 != num_eq2)) and ((eq1 < eq2) or (eq1 == eq2 and num_eq1 < num_eq2))
                          for dia in Dias
                          for horario in Horarios[dia]) + lpSum(x[(eq2, num_eq2, eq1, num_eq1, dia, horario)]
                          for eq2 in Categorias
                          for num_eq2 in range(1, Equipos_categoria[eq2] + 1)
                          if (Categorias.index(eq2) <= Categorias.index(eq1)) and ((eq2 != eq1) or (num_eq2 != num_eq1)) and ((eq2 < eq1) or (eq2 == eq1 and num_eq2 < num_eq1))
                          for dia in Dias
                          for horario in Horarios[dia])) == 1

# 4.2) Restricciones de obligatoriedad del viernes y lunes
for dia in ['Viernes', 'Lunes']:
    problema += lpSum(x[(eq1, num_eq1, eq2, num_eq2, dia, horario)]
                      for eq1 in Categorias
                      for num_eq1 in range(1, Equipos_categoria[eq1] + 1)
                      for eq2 in Categorias
                      for num_eq2 in range(1, Equipos_categoria[eq2] + 1)
                      if (Categorias.index(eq1) <= Categorias.index(eq2)) and ((eq1 != eq2) or (num_eq1 != num_eq2)) and ((eq1 < eq2) or (eq1 == eq2 and num_eq1 < num_eq2))
                      for horario in Horarios[dia]) >= 1
    
# 4.3) Asignar penalización horaria
for dia in Dias:
    for horario in Horarios[dia]:
        penalizacion_coicidencias = lpSum(x[(eq1, num_eq1, eq2, num_eq2, dia, horario)]
                  for eq1 in Categorias
                  for num_eq1 in range(1, Equipos_categoria[eq1] + 1)
                  for eq2 in Categorias
                  for num_eq2 in range(1, Equipos_categoria[eq2] + 1)
                  if (Categorias.index(eq1) <= Categorias.index(eq2)) and ((eq1 != eq2) or (num_eq1 != num_eq2)) and ((eq1 < eq2) or (eq1 == eq2 and num_eq1 < num_eq2))
                  )
        # Obtener los coeficientes y variables asociadas
        coeficientes_variables = [(var, coef) for (coef, var) in penalizacion_coicidencias.items() if var > 1]

# Imprimir la lista de coeficientes y variables asociadas
print(coeficientes_variables)

for dia in Dias:
    for horario in Horarios[dia]:
        problema += lpSum(y[(dia, horario)]) == lpSum(x[(eq1, num_eq1, eq2, num_eq2, dia, horario)]
                                                      for eq1 in Categorias
                                                      for num_eq1 in range(1, Equipos_categoria[eq1] + 1)
                                                      for eq2 in Categorias
                                                      for num_eq2 in range(1, Equipos_categoria[eq2] + 1)
                                                      if (Categorias.index(eq1) <= Categorias.index(eq2)) and ((eq1 != eq2) or (num_eq1 != num_eq2)) and ((eq1 < eq2) or (eq1 == eq2 and num_eq1 < num_eq2)))

# Resolver el problema
status = problema.solve()
print(LpStatus[status])

# Imprimir el valor de la función objetivo
print("Valor de la función objetivo:", value(problema.objective))

# Mostrar resultados
for v in problema.variables():
    if value(v) > 0:
        print(f"{v.name}: {value(v)}")

[]
Optimal
Valor de la función objetivo: 8.776000000000003
Partido_('A',_1,_'B',_5,_'Domingo',_20): 1.0
Partido_('A',_2,_'A',_3,_'Sábado',_20): 1.0
Partido_('B',_1,_'B',_7,_'Sábado',_20): 1.0
Partido_('B',_11,_'C',_3,_'Sábado',_20): 1.0
Partido_('B',_2,_'B',_4,_'Domingo',_20): 1.0
Partido_('B',_3,_'C',_4,_'Sábado',_20): 1.0
Partido_('B',_6,_'B',_9,_'Domingo',_20): 1.0
Partido_('B',_8,_'B',_10,_'Domingo',_20): 1.0
Partido_('C',_1,_'C',_6,_'Lunes',_20): 1.0
Partido_('C',_2,_'C',_5,_'Viernes',_20): 1.0
PenalizacionHoraria_('Domingo',_20): 4.0
PenalizacionHoraria_('Lunes',_20): 1.0
PenalizacionHoraria_('Sábado',_20): 4.0
PenalizacionHoraria_('Viernes',_20): 1.0


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

In [ ]:
#Respuesta


#Diseño
- ¿Que técnica utilizo? ¿Por qué?

In [ ]:
#Respuesta